In [1]:
from data_loader import get_mix_instruct

prompts, references, ds_name = get_mix_instruct("train", 21000)

Dataset: mix-instruct_train_21000 not found in cache, downloading from huggingface 🏃
Dataset: mix-instruct_train_21000 loaded and saved to cache ✅


In [ ]:
from utility_functions.delift_se import get_delift_se_utility

utility, utility_name = get_delift_se_utility(prompts, references, ds_name)

In [ ]:
from subset import create_subset

subset, subset_name = create_subset(utility, utility_name)

In [ ]:
from inference import generate_responses

responses, generation_name = generate_responses(prompts[:100], "microsoft/Phi-3-mini-128k-instruct", ds_name, 'cuda:3', batch_size=64)

Generation: mix-instruct_train_21000_Phi-3-mini-128k-instruct not found in cache, generating responses 🏃


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Generating responses:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import torch
torch.cuda.empty_cache()